In [1]:
from bs4 import BeautifulSoup
from utils import get_chromedriver, get_icos_list_by_category, adjust_date, eval_duration, scrape_info_icomarks
from utils import extract_scaping_icomarks, summary_stats, format_columns
import requests
import pandas as pd
from urllib.parse import urljoin
from soup2dict import convert
import time
from timeit import default_timer as timer
import datetime
import numpy as np
import re
import pandas as pd
import os
import joblib
import sys
from thefuzz import fuzz
from thefuzz import process
import yfinance as yf

In [3]:
CHROMEDRIVER_PATH = r"C:\Users\Alessandro Bitetto\Downloads\UniPV\ICOs\WebDriver\chromedriver"

In [12]:
# set folders
CHECKPOINT_FOLDER = '.\\Checkpoints'
RESULTS_FOLDER = '.\\Results'
ICOMARKS_CATEG_FOLDER=os.path.join(CHECKPOINT_FOLDER, 'Icomarks_category')
ICOMARKS_FOLDER=os.path.join(CHECKPOINT_FOLDER, 'Icomarks')

if not os.path.exists(CHECKPOINT_FOLDER):
    os.makedirs(CHECKPOINT_FOLDER)
if not os.path.exists(RESULTS_FOLDER):
    os.makedirs(RESULTS_FOLDER)
if not os.path.exists(ICOMARKS_CATEG_FOLDER):
    os.makedirs(ICOMARKS_CATEG_FOLDER)
if not os.path.exists(ICOMARKS_FOLDER):
    os.makedirs(ICOMARKS_FOLDER)

In [13]:
NEW_DOMAIN = ".ai"      # replace the old domani ".com", changed after first run

## Get ICOs url

In [21]:
MAIN_PAGE = "https://icomarks.com/"                # to be added to single ICO url
CATEGORY_PAGE = "https://icomarks.com/icos/"       # used to query the category to be downloaded

# get html
page = requests.get(CATEGORY_PAGE.replace(".com", NEW_DOMAIN))
soup = BeautifulSoup(page.content, 'html.parser')

# extract list of categories
tag = soup.find_all('div', class_="icoTop__selects", recursive=True)
conv_dict = convert(tag)
while conv_dict['div'][0]['@class'][0] != 'icoTop__selects':
    conv_dict = conv_dict['div'][0]
category_list = conv_dict['div'][0]['form'][0]['select'][0]['option']
category = pd.DataFrame([(v['@value'], v['#text']) for v in category_list if '@value' in v.keys()],
                        columns =['url_ref', 'Category'])
category[['Category', 'Count']] = category['Category'].str.split('(', 1, expand=True)
category['Count'] = category['Count'].apply(lambda x: int(x.replace(')', '')))
display(category)

platform_list=conv_dict['div'][0]['form'][0]['select'][1]['option']
platform = pd.DataFrame([(v['@value'], v['#text']) for v in platform_list if '@value' in v.keys()],
                        columns =['url_ref', 'Platform'])

status_list=conv_dict['div'][0]['form'][0]['select'][2]['option']
status = pd.DataFrame([(v['@value'], v['#text']) for v in status_list if '@value' in v.keys()],
                        columns =['url_ref', 'Status'])

C:\Users\Alessandro Bitetto\AppData\Local\Temp\ipykernel_13468\1522410269.py:16: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  category[['Category', 'Count']] = category['Category'].str.split('(', 1, expand=True)


,url_ref,Category,Count
0,artificial-intelligence,AI,540
1,art,Art,104
2,banking,Banking,647
3,big-data,Big Data,412
4,business-services,Business,1346
5,charity,Charity,159
6,communication,Communication,451
7,cryptocurrency,Cryptocurrency,3028
8,defi,DeFi,557
9,education,Education,219


In [24]:
# apply category in search query and get ICO list
SPLIT_BY = 'platform'   # 'status' or 'platform', used to split download for categories with more than 1500 entries
RELOAD_PKL = True

cat_list = pd.DataFrame(columns=['Category', 'url', 'NViews', 'VerifiedEmailDummy', 'IsSTODummy', 'IsIEODummy',
                                 'Status', 'StartDate', 'EndDate'])

download_date=datetime.datetime.now().strftime("%d/%m/%Y")
tot_time = 0
for index, row in category.iterrows():
    
    url_categ = row['url_ref']
    expected_count = row['Count']
    categ = row['Category']
    count = row['Count']
    
    pkl_path=os.path.join(ICOMARKS_CATEG_FOLDER, url_categ+'.pkl').replace('|', '')
    
    print(f'\n- Downloading: {categ}  ({expected_count} expected) {index + 1} / {len(category)}')
    
    try:
        if not RELOAD_PKL or not os.path.exists(pkl_path):
            start = timer()
            # decide if splitting the download by filtering each platform (longer list may crash when scrolling down)
            if count > 1500:
                if SPLIT_BY == 'status':
                    split_df = status
                elif SPLIT_BY == 'platform':
                    split_df = platform

                temp_list = pd.DataFrame()
                for i, filter_row in split_df.iterrows():

                    print(f'   - Scrolling down, Parsing info and Downloading html for each split... {i+1} / {len(split_df)}', end='\r')

                    filter_val = filter_row['url_ref']
                    url_filter = f'?{SPLIT_BY}={filter_val}&whitelist=&kyc=&bounty=&mvp=&email_confirmed='
                    url = urljoin(CATEGORY_PAGE.replace(".com", NEW_DOMAIN), url_categ+url_filter)

                    tt = get_icos_list_by_category(url, categ, CHROMEDRIVER_PATH, MAIN_PAGE, NEW_DOMAIN, split=True)
                    temp_list = pd.concat([temp_list, tt])
                temp_list = temp_list.drop_duplicates()
                print('\n')
            else:
                url = urljoin(CATEGORY_PAGE.replace(".com", NEW_DOMAIN), url_categ)
                temp_list = get_icos_list_by_category(url, categ, CHROMEDRIVER_PATH, MAIN_PAGE, NEW_DOMAIN)
            eval_time = datetime.timedelta(seconds=round(timer()-start)).total_seconds()

            # save pkl
            joblib.dump({'temp_list': temp_list,
                         'eval_time': eval_time}, pkl_path, compress=('lzma', 3))
        else:
            print('   - Scrolling down...Reloaded')
            print('   - Downloading html...Reloaded')
            print('   - Parsing info...Reloaded', end='  ')
            rr = joblib.load(pkl_path)
            temp_list = rr['temp_list']
            eval_time = rr['eval_time']

        tot_time += eval_time
        temp_list['ListDownloadedOn']=download_date
        cat_list = pd.concat([cat_list, temp_list])

        if temp_list.shape[0] != expected_count:
            print('   ####### warning, expected number of elements (' + str(expected_count) + ') mismatch. Found ' + str(temp_list.shape[0]))
        else:
            print('OK')

        print('     Elapsed time:', str(datetime.timedelta(seconds=round(eval_time))))
    except:
        print('######## ERROR')
        
# save results
cat_list.to_csv(os.path.join(RESULTS_FOLDER,'01a_ICOmarks_ico_list_raw.csv'), index=False, sep=';')    
    
print('\n\nTotal elapsed time:', str(datetime.timedelta(seconds=round(tot_time))))
print('\nData saved in ', os.path.join(RESULTS_FOLDER,'01a_ICOmarks_ico_list_raw.csv'))


- Downloading: AI   (540 expected) 1 / 33
   - Scrolling down...Reloaded
   - Downloading html...Reloaded
   - Parsing info...Reloaded  OK
     Elapsed time: 0:01:38

- Downloading: Art   (104 expected) 2 / 33
   - Scrolling down...Reloaded
   - Downloading html...Reloaded
   - Parsing info...Reloaded  OK
     Elapsed time: 0:00:22

- Downloading: Banking   (647 expected) 3 / 33
   - Scrolling down...Reloaded
   - Downloading html...Reloaded
   - Parsing info...Reloaded     ####### warning, expected number of elements (647) mismatch. Found 160
     Elapsed time: 0:00:28

- Downloading: Big Data   (412 expected) 4 / 33
   - Scrolling down...OK
   - Downloading html...OK
   - Parsing info...OK
     Elapsed time: 0:01:12

- Downloading: Business   (1346 expected) 5 / 33
   - Scrolling down...OK
   - Downloading html...OK
   - Parsing info...OK
     Elapsed time: 0:04:09

- Downloading: Charity   (159 expected) 6 / 33
   - Scrolling down...OK
   - Downloading html...OK
   - Parsing info..

### Check downloaded list and remove duplicates

In [5]:
cat_list=pd.read_csv(os.path.join(RESULTS_FOLDER,'01a_ICOmarks_ico_list_raw.csv'), sep=';')

cat_list.drop_duplicates(inplace=True)

# adjust dates
cat_list['StartDate']=cat_list['StartDate'].map(adjust_date)
cat_list['EndDate']=cat_list['EndDate'].map(adjust_date)

# find url with multiple entries (due to IEO/STO) and keep all categories and minimum start date and max end date
multiple_url=cat_list[['url', 'NViews']].drop_duplicates()['url'].value_counts().to_frame().reset_index().query('url > 1')['index']
if len(multiple_url) > 0:
    
    print('\n-- Url with multiple entries found. Keeping single information only')
    new_df=pd.DataFrame(columns=cat_list.columns)
    for t_url in multiple_url:
        t_df=cat_list[cat_list['url']==t_url].copy()

        status=t_df['Status'].value_counts().index[0]
        if t_df['Status'].nunique() > 1:
            u_val=t_df['Status'].unique()
            if 'Ended' in u_val:
                status='Ended'
            if 'Active' in u_val:
                status='Active'
            print(f"    - {t_url}: Multiple status found: {u_val}. Keeping '{status}'")

        try:
            start_date=pd.to_datetime(t_df['StartDate'].loc[lambda x : x != 'TBA'], infer_datetime_format=True).min().strftime('%d %b %Y')
        except:
            start_date=t_df['StartDate'].unique()[0]
        try:
            end_date=pd.to_datetime(t_df['EndDate'].loc[lambda x : x != 'TBA'], infer_datetime_format=True).max().strftime('%d %b %Y')
        except:
            end_date=t_df['EndDate'].unique()[0]

        add_df=pd.DataFrame({'Category': t_df['Category'].unique(),
                            'url': t_url,
                            'NViews': t_df['NViews'].max(),
                            'VerifiedEmailDummy': t_df['VerifiedEmailDummy'].max(),
                            'IsSTODummy': t_df['IsSTODummy'].max(),
                            'IsIEODummy': t_df['IsIEODummy'].max(),
                            'Status': status,
                            'StartDate': start_date,
                            'EndDate': end_date,
                            'ListDownloadedOn': t_df['ListDownloadedOn'].values[0]})

        new_df=pd.concat([new_df, add_df])
        
    cat_list=cat_list[~cat_list['url'].isin(multiple_url)]
    cat_list=pd.concat([cat_list, new_df]) 

# get dummy for category
cat_list['Category']=cat_list['Category'].str.replace(' ', '')
cat_dummy=pd.concat([cat_list['url'], pd.get_dummies(cat_list['Category'], drop_first=False, prefix='Category', prefix_sep='')], axis=1)
cat_dummy=cat_dummy.groupby('url').sum()
cat_dummy.columns=cat_dummy.columns+'Dummy'
if cat_dummy.max().max() != 1:
    print('\n ### "Category" dummy variable has value greater than 1')
cat_dummy.reset_index(inplace=True)

# evaluate duration
cat_list.loc[cat_list['url'] == 'https://icomarks.ai/ico/kaizen-coin', 'StartDate']='18 Aug 2017'
cat_list.loc[cat_list['url'] == 'https://icomarks.ai/ico/curveblock', 'EndDate']='31-mar-19'
cat_list.loc[cat_list['url'] == 'https://icomarks.ai/ico/0chain', 'EndDate']='19-feb-18'
cat_list.loc[cat_list['url'] == 'https://icomarks.ai/ico/hunibit', 'StartDate']='21 apr 2019'
cat_list.loc[cat_list['url'] == 'https://icomarks.ai/ico/hunibit', 'EndDate']='03 may 2019'
cat_list.loc[cat_list['url'] == 'https://icomarks.ai/ico/clearaid', 'EndDate']='01 may 2019'
cat_list.loc[cat_list['url'] == 'https://icomarks.ai/ico/ultrashares', 'StartDate']='23 apr 2018'
cat_list.loc[cat_list['url'] == 'https://icomarks.ai/ico/ultrashares', 'EndDate']='30 jun 2018'
cat_list.loc[cat_list['url'] == 'https://icomarks.ai/ico/dentix', 'StartDate']='01-mar-18'
cat_list.loc[cat_list['url'] == 'https://icomarks.ai/ico/eos', 'EndDate']='04 jun 2018'
cat_list.loc[cat_list['url'] == 'https://icomarks.ai/ico/mundus', 'StartDate']='31 aug 2017'
cat_list.loc[cat_list['url'] == 'https://icomarks.ai/ico/mundus', 'EndDate']='30 oct 2017'
cat_list.loc[cat_list['url'] == 'https://icomarks.ai/ico/vr-park', 'StartDate']='24 aug 2019'
cat_list.loc[cat_list['url'] == 'https://icomarks.ai/ico/vr-park', 'EndDate']='17 apr 2020'
cat_list.loc[cat_list['url'] == 'https://icomarks.ai/ico/goldminecoin', 'EndDate']='15 mar 2018'
cat_list.loc[cat_list['url'] == 'https://icomarks.ai/ico/horsechain', 'EndDate']='14 Jul 2019'
cat_list['LogDurationDays']=cat_list.apply(eval_duration, axis=1)
move_col = cat_list.pop('LogDurationDays')
cat_list.insert(cat_list.columns.get_loc("EndDate")+1, 'LogDurationDays', move_col)
check_error=cat_list[cat_list['LogDurationDays'] < 0][['url', 'Status', 'StartDate', 'EndDate']].drop_duplicates()
if len(check_error):
    print(f'\n-- {len(check_error)} rows with error in "LogDurationDays":')
    display(check_error['Status'].value_counts().to_frame())
    check_error.to_csv(os.path.join(RESULTS_FOLDER, '01b_ICOmarks_ico_list_adjusted_DurationError.csv'), index=False, sep=';')
    print('Log saved in ', os.path.join(RESULTS_FOLDER, '01b_ICOmarks_ico_list_adjusted_DurationError.csv'))
    

# create final dataset
cat_list=cat_list.drop(columns='Category').drop_duplicates()
cat_list=cat_list.merge(cat_dummy, on='url', how='left')
move_col = cat_list.pop('ListDownloadedOn')
cat_list.insert(cat_list.columns.get_loc("url")+1, 'ListDownloadedOn', move_col)

if cat_list['url'].nunique() != cat_list.shape[0]:
    print('\n ##### Unique urls do not match number of rows')

print('\n-- Total ICOs found:', cat_list['url'].nunique())
display(cat_list['Status'].value_counts().to_frame())
    
# save csv
cat_list.to_csv(os.path.join(RESULTS_FOLDER, '01b_ICOmarks_ico_list_adjusted.csv'), index=False, sep=';')
print('\nData saved in ', os.path.join(RESULTS_FOLDER, '01b_ICOmarks_ico_list_adjusted.csv'))


-- Url with multiple entries found. Keeping single information only
    - https://icomarks.ai/ico/ins: Multiple status found: ['Trading' 'Ended']. Keeping 'Ended'
    - https://icomarks.ai/ico/unifox: Multiple status found: ['Pre-Sale Ended' 'Ended']. Keeping 'Ended'

-- Total ICOs found: 8348


,Status
Ended,4998
Upcoming,1975
Trading,697
Pre-Sale Ended,444
Active,164
Pre-Sale,70



Data saved in  .\Results\01b_ICOmarks_ico_list_adjusted.csv


### Compare with previous download

In [14]:
new = pd.read_csv(os.path.join(RESULTS_FOLDER, '01b_ICOmarks_ico_list_adjusted.csv'), sep=';')
old = pd.read_csv(os.path.join(RESULTS_FOLDER, 'OLD 01b_ICOmarks_ico_list_adjusted.csv'), sep=';')
old['url'] = old['url'].str.replace(".com", NEW_DOMAIN, regex=False)

url_common = set(new['url']) & set(old['url'])
url_new_only = set(new['url']) - set(old['url'])
url_old_only = set(old['url']) - set(new['url'])
print('-- Compare with previous download:')
print(f'  {len(url_common)} common url found')
print(f'  {len(url_old_only)} old url missing')
print(f'  {len(url_new_only)} new url found')
new[new['url'].isin(url_new_only)].to_csv(os.path.join(RESULTS_FOLDER, '01b_ICOmarks_ico_list_new_only.csv'), index=False, sep=';')
print('      *** List with new url only saved in ', os.path.join(RESULTS_FOLDER, '01b_ICOmarks_ico_list_new_only.csv'))

common_diff = pd.DataFrame()
for url in url_common:
    new_t = new[new['url']==url]
    old_t = old[old['url']==url]
    
    diff=''
    
    if (new_t['IsSTODummy'].values[0] != old_t['IsSTODummy'].values[0] or
        new_t['IsIEODummy'].values[0] != old_t['IsIEODummy'].values[0] or
        new_t['VerifiedEmailDummy'].values[0] != old_t['VerifiedEmailDummy'].values[0]):
        diff += '| different Dummy'

    if new_t['Status'].values[0] != old_t['Status'].values[0]:
        diff += '| different Status'
    
    if (new_t['LogDurationDays'].values[0] != old_t['LogDurationDays'].values[0] and
        ~new_t['LogDurationDays'].isna().values[0] and
        ~old_t['LogDurationDays'].isna().values[0]):
        diff += '| different Date'
    
    if len(diff) > 0:
        common_diff = pd.concat([common_diff, pd.DataFrame({'url': url, 'mismatch': diff[1:]}, index=[0])])
        
print(f'\n\n-- {len(common_diff)} mismatch on common url found:')
display(common_diff['mismatch'].value_counts().to_frame())

common_diff.to_csv(os.path.join(RESULTS_FOLDER, '01b_ICOmarks_ico_list_adjusted_difference_with_OLD.csv'), index=False, sep=';')
print('\nLog saved in ', os.path.join(RESULTS_FOLDER, '01b_ICOmarks_ico_list_adjusted_difference_with_OLD.csv'))

cat_list = pd.concat([old[old['url'].isin(url_old_only)], new[new['url'].isin(url_common | url_new_only)]]).reset_index(drop=True)
if len(cat_list) != cat_list['url'].nunique():
    print('######### duplicates in "cat_list"')
print('\n-- New information for ICOs is kept and missing old ICOs are added')
print('\n-- Total ICOs found:', cat_list['url'].nunique())
display(cat_list['Status'].value_counts().to_frame())

# save csv
cat_list.to_csv(os.path.join(RESULTS_FOLDER, '01b_ICOmarks_ico_list_merged.csv'), index=False, sep=';')
print('\nData saved in ', os.path.join(RESULTS_FOLDER, '01b_ICOmarks_ico_list_merged.csv'))

-- Compare with previous download:
  8032 common url found
  247 old url missing
  316 new url found
      *** List with new url only saved in  .\Results\01b_ICOmarks_ico_list_new_only.csv


-- 257 mismatch on common url found:


,mismatch
different Date,199
different Status| different Date,37
different Dummy,10
different Status,7
different Dummy| different Status| different Date,2
different Dummy| different Status,1
different Dummy| different Date,1



Log saved in  .\Results\01b_ICOmarks_ico_list_adjusted_difference_with_OLD.csv

-- New information for ICOs is kept and missing old ICOs are added

-- Total ICOs found: 8595


,Status
Ended,5123
Upcoming,2054
Trading,727
Pre-Sale Ended,454
Active,167
Pre-Sale,70



Data saved in  .\Results\01b_ICOmarks_ico_list_merged.csv


## Scrape information from url

In [11]:
URL_ROOT='https://icomarks.com/ico/'    # will be removed from url to save pickle in ICOMARKS_FOLDER
RELOAD_PKL=True
SKIP_MISSING=False     # if True skip attempt to scrape missing pickles

if not os.path.exists(ICOMARKS_FOLDER):
    os.makedirs(ICOMARKS_FOLDER)

cat_list=pd.read_csv(os.path.join(RESULTS_FOLDER, '01b_ICOmarks_ico_list_merged.csv'), sep=';')
url_mismatch=pd.read_csv(os.path.join(RESULTS_FOLDER, '01b_ICOmarks_ico_list_adjusted_difference_with_OLD.csv'), sep=';')['url']

In [12]:
scrape_df=pd.DataFrame()
for index, row in cat_list.iterrows():
    
    print(f'- Scraping: {str(index + 1)} / {len(cat_list)}   last interaction: {datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")}', end='\r')
    
    url=row['url']
    save_path=os.path.join(ICOMARKS_FOLDER, url.replace(URL_ROOT.replace(".com", NEW_DOMAIN), '')+'.json').replace('|', '')
    
    if not RELOAD_PKL or not os.path.exists(save_path) or url in url_mismatch:
    
        if SKIP_MISSING and not os.path.exists(save_path):
            add_row=pd.DataFrame({'url': url, 'ScrapeStatus': 'ERROR'}, index=[0])
            scrape_df=pd.concat([scrape_df, add_row])
            continue
    
        try:
            start = timer()
            add_row=scrape_info_icomarks(url=url, chromedriver_path=CHROMEDRIVER_PATH, skip_social=False, skip_price=False)
            add_row.insert(1, 'ScrapeStatus', 'OK')
            add_row['PklPath']=save_path
            add_row['TotTimeSec']=datetime.timedelta(seconds=round(timer()-start)).total_seconds()
            add_row.to_json(save_path, orient='table')
        except:
            add_row=pd.DataFrame({'url': url, 'ScrapeStatus': 'ERROR'}, index=[0])
    
    else:
        add_row=pd.read_json(save_path, orient='table')
        # re-format nested dataframe from json schema
        add_row['InfoBlock']=[pd.DataFrame(add_row['InfoBlock'][0])]
        if 'TeamBlock' in add_row.columns:
            add_row['TeamBlock']=[pd.DataFrame(add_row['TeamBlock'][0])]
        if 'SocialBlock' in add_row.columns:
            social_df=pd.DataFrame(add_row['SocialBlock'][0][0]['stats'])
            series_dict={}
            for k in add_row['SocialBlock'][0][0]['timeseries'].keys():
                series_dict[k]=pd.DataFrame(add_row['SocialBlock'][0][0]['timeseries'][k])
            add_row['SocialBlock']=[[{'stats': social_df, 'timeseries': series_dict}]]
        if 'MarketPriceSeries' in add_row.columns:
            add_row['MarketPriceSeries']=[pd.DataFrame(add_row['MarketPriceSeries'][0])]
        
    scrape_df=pd.concat([scrape_df, add_row])

scrape_df.reset_index(drop=True, inplace=True)
display(scrape_df['ScrapeStatus'].value_counts().to_frame())
   
print('\n\nTotal elapsed time:', str(datetime.timedelta(seconds=round(scrape_df['TotTimeSec'].sum()))))

# save
scrape_df['url'] = scrape_df['url'].str.replace(".com", NEW_DOMAIN, regex=False)
pkl_path=os.path.join(CHECKPOINT_FOLDER, 'scrape_df_raw.pkl')
joblib.dump(scrape_df, pkl_path, compress=('lzma', 3))
print(f'\nData saved in {pkl_path}')

,ScrapeStatus
OK,8595




Total elapsed time: 1 day, 11:08:23

Data saved in .\Checkpoints\scrape_df_raw.pkl


## Format scraped information and save final dataset

#### Download FX rates from Finance.yahoo.com and Investing.com

In [14]:
# data from Investing.com must be downloaded manually. See utils.py

for tick in ['ETH-USD', 'BNB-USD', 'USDT-USD', 'BTC-USD', 'XLM-USD', 'TRX-USD', 'BUSD-USD', 'KRW-USD', 'NEO-USD', 'MATIC-USD',
             'USDC-USD', 'WAVES-USD', 'BTS-USD', 'VET-USD', 'ICX-USD', 'LNC-USD', 'WETH-USD', 'DOT-USD', 'AVAX-USD', 'AV-USD']:
    
    # download new data
    data = yf.download(tick, interval='1mo', period='max')
    data = data.reset_index()

    # reload previous data and merge
    dd = pd.read_csv(os.path.join('.\\Data and papers\\FX', tick+'.csv'), parse_dates=['Date'])
    common_date = set(data['Date']) & set(dd['Date'])
    recover_date = set(dd['Date']) - set(data['Date'])
    merge_data = pd.concat([dd[dd['Date'].isin(recover_date)], data])
    merge_data = merge_data.sort_values(by='Date', ascending=True)

    merge_data.to_csv(os.path.join('.\\Data and papers\\FX', tick+'.csv'), index=False)
    
print(f'\nData saved in ', '.\\Data and papers\\FX')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******


Data saved in  .\Data and papers\FX


In [5]:
pkl_path=os.path.join(CHECKPOINT_FOLDER, 'scrape_df_raw.pkl')
scrape_df=joblib.load(pkl_path)

start=timer()

#### extract from raw data
print('----######   Extracting information from raw scraped data   ######----\n')
format_df=extract_scaping_icomarks(scrape_df).reset_index(drop=True)

pkl_path=os.path.join(CHECKPOINT_FOLDER, 'scrape_df_extracted.pkl')
joblib.dump(format_df, pkl_path, compress=('lzma', 3))
print(f'\n   - Data saved in {pkl_path}')


#### format nested column and merge with cat_list dataset
cat_list=pd.read_csv(os.path.join(RESULTS_FOLDER, '01b_ICOmarks_ico_list_merged.csv'), sep=';')
print('\n\n\n----######   Formatting columns   ######----')
if len(cat_list) != len(format_df):
    raise ValueError('\n\n ########### Error: "cat_list" and "format_df" must have same rows')
format_df_rows=format_df.shape[0]
format_df=format_columns(format_df, cat_list=cat_list, format_df_rows=format_df_rows, results_folder=RESULTS_FOLDER)
print('\n\n\n----######   Merging with category dataset   ######----')
cat_list_rows=cat_list.shape[0]
final_df=cat_list.copy().merge(format_df, on='url', how='left')
final_df['WebsiteUrl']=final_df['WebsiteUrl'].str.replace('?utm_source=icomarks', '', regex=False)
display(final_df['Status'].value_counts().to_frame())
print(f'   - Total rows: {len(final_df)}')
if final_df.shape[0] != cat_list_rows:
    print('########## "final_df" expected rows do not match')
# save file
save_path=os.path.join(RESULTS_FOLDER, '01g_ICOmarks_ico_list_scraped_formatted.csv')
save_path_pkl=os.path.join(CHECKPOINT_FOLDER, 'formatted_df.pkl')
final_df.to_csv(save_path, index=False, sep=';')
final_df.to_pickle(save_path_pkl, protocol=-1)
print(f'\n   - Data saved in {save_path}')
print(f'   - Pickle saved in {save_path_pkl}')
# save stats
save_path=os.path.join(RESULTS_FOLDER, '01g_ICOmarks_ico_list_scraped_formatted_Stats.csv')
print(f'   - Stats saved in {save_path}')
summary_stats(final_df).to_csv(save_path, index=False, sep=';')

print('\n\nTotal elapsed time:', str(datetime.timedelta(seconds=round(timer()-start))))

----######   Extracting information from raw scraped data   ######----

   - Processing 8595 / 8595
   - Data saved in .\Checkpoints\scrape_df_extracted.pkl



----######   Formatting columns   ######----

** Formatting "FundRaised"

** Formatting "Country"
- Mapped countries with low accuracy:


,Country,Country_adj,country,accuracy
152,Grand Cayman,Grand Cayman,Cayman Islands,73
45,Singarope,Singarope,Singapore,78
235,Singapura,Singapura,Singapore,78
155,Malte,Malte,Malta,80
172,Nederland,Nederland,Netherlands,80
224,Melta,Melta,Malta,80
95,Brasil,Brasil,Brazil,83
154,Сanada,Сanada,Canada,83
264,Tunis,Tunis,Tunisia,83
177,St Vincent,St Vincent,Saint Vincent and the Grenadines,86



** Formatting "SocialMedia"
- Counts for SocialMedia dummy:


,val,count
10,Twitter,7963
9,Telegram,7185
2,Facebook,6115
6,Medium,4718
12,Youtube,4105
0,Bitcointalk,3846
7,Reddit,3643
3,Github,3269
5,Linkedin,1250
4,Instagram,1154



** Formatting "ICOPrice", "IEOPrice", "STOPrice"
- Errors when parsing "ICOPrice", "IEOPrice", "STOPrice":


...

- Taking closest available FX rate for 253 rows. Currency: ['ETH' 'KRW']
- 7359 rows remaing in "only_currency_df" (with Hard/Soft Cap in currency)
- All 4 rows in "both_df" (with Hard/Soft Cap in currency AND token) skipped because of mismatch

- Hard/Soft Cap available for 7834 entries
- Price log saved in .\Results\01c_ICOmarks_formatted_HardSoftCap_log.csv

** Formatting "Platform"
- Unique platform found: 136
- Unique list saved in .\Results\01d_ICOmarks_formatted_Platform_unique.csv

** Formatting "TokenAvailForSale" and "TokenTotSupply"
- Errors when parsing "TokenAvailForSale":


,error
,8595


- Error log saved in .\Results\01e_ICOmarks_formatted_TokenAvailForSale_error_log.csv

** Formatting "AcceptedCurr"
- Unique AcceptedCurr found: 274
- Unique list saved in .\Results\01f_ICOmarks_formatted_AcceptedCurr_unique.csv



----######   Merging with category dataset   ######----


,Status
Ended,5123
Upcoming,2054
Trading,727
Pre-Sale Ended,454
Active,167
Pre-Sale,70


   - Total rows: 8595

   - Data saved in .\Results\01g_ICOmarks_ico_list_scraped_formatted.csv
   - Pickle saved in .\Checkpoints\formatted_df.pkl
   - Stats saved in .\Results\01g_ICOmarks_ico_list_scraped_formatted_Stats.csv
    - 2 columns excluded: Platform, AcceptedCurr


Total elapsed time: 0:04:38


## Extract Market Price and Social Users list

In [10]:
pkl_path=os.path.join(CHECKPOINT_FOLDER, 'scrape_df_raw.pkl')
scrape_df=joblib.load(pkl_path)

In [13]:
start = timer()
price_df=pd.DataFrame()
social_df=pd.DataFrame()
for index, row in scrape_df.iterrows():
    
    # price
    if row['MarketPriceSeriesStatus'] == 'DOWNLOADED':
        tt=row['MarketPriceSeries'].copy()
        tt.insert(0, 'url', row['url'])
        price_df=pd.concat([price_df, tt])
    
    # social users
    if row['SocialSeriesStatus'] == 'DOWNLOADED':
        tt=pd.DataFrame()
        for social_name, social_data in row['SocialBlock'][0]['timeseries'].items():
            df=social_data.copy()
            df.insert(0, 'Social', social_name)
            df.insert(0, 'url', row['url'])
            tt=pd.concat([tt, df])
        social_df=pd.concat([social_df, tt])
     
tot_url=price_df['url'].nunique()
print(f'- Downloaded Market Price: {tot_url}  Total rows: {len(price_df)}')
save_path=os.path.join(RESULTS_FOLDER, '01h_ICOmarks_market_price_series.csv')
save_path_pkl=os.path.join(CHECKPOINT_FOLDER, 'ICOmarks_market_price_series.pkl')
price_df.to_csv(save_path, index=False, sep=';')
price_df.to_pickle(save_path_pkl, protocol=-1)
print(f'   - Data saved in {save_path}')
print(f'   - Pickle saved in {save_path_pkl}')


tot_url=social_df['url'].nunique()
print(f'\n- Downloaded Social Users: {tot_url}  Total rows: {len(social_df)}')
save_path=os.path.join(RESULTS_FOLDER, '01h_ICOmarks_social_users_series.csv')
save_path_pkl=os.path.join(CHECKPOINT_FOLDER, 'ICOmarks_social_users_series.pkl')
# social_df.to_csv(save_path, index=False, sep=';')
social_df.to_pickle(save_path_pkl, protocol=-1)
print(f'   - Data saved in {save_path}')
print(f'   - Pickle saved in {save_path_pkl}')
print('\n\nTotal elapsed time:', str(datetime.timedelta(seconds=round(timer()-start))))

- Downloaded Market Price: 724  Total rows: 335746
   - Data saved in .\Results\01h_ICOmarks_market_price_series.csv
   - Pickle saved in .\Checkpoints\ICOmarks_market_price_series.pkl

- Downloaded Social Users: 6415  Total rows: 7276539
   - Data saved in .\Results\01h_ICOmarks_social_users_series.csv
   - Pickle saved in .\Checkpoints\ICOmarks_social_users_series.pkl


Total elapsed time: 0:19:02
